Connected to Python 3.12.0

### Extração dos dados de exportacao de vinhos 

In [127]:
import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import geopandas as gpd
import matplotlib.pyplot as plt

In [128]:
# csv leitura de dados de exportacao
data_vinhos = pd.read_csv("embrapa_dt/ExpVinho.csv", sep=";")

data_vinhos.head(4)

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0


### Validando os dados

In [129]:
# etender os formatos da tabela 
print(data_vinhos.info())
print("\n")
# entender se os países se repetem em nome se a sua quantidade for a mesma do shape 0 
print(f"se os países se repetem tamanho tabela (tamanho, comprimento) {data_vinhos.shape}, países {len(data_vinhos["País"].unique())}")
print("\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Columns: 110 entries, Id to 2023.1
dtypes: int64(109), object(1)
memory usage: 117.9+ KB
None


se os países se repetem tamanho tabela (tamanho, comprimento) (137, 110), países 137




In [130]:
# Exibir o número de valores nulos em cada coluna
print(data_vinhos.isnull().sum())
print("\n")
# Exibir as linhas que têm valores nulos index []
print("\nExibir linhas que tem valores nulos index []")
print(data_vinhos[data_vinhos.isnull().any(axis=1)])

Id        0
País      0
1970      0
1970.1    0
1971      0
         ..
2021.1    0
2022      0
2022.1    0
2023      0
2023.1    0
Length: 110, dtype: int64



Exibir linhas que tem valores nulos index []
Empty DataFrame
Columns: [Id, País, 1970, 1970.1, 1971, 1971.1, 1972, 1972.1, 1973, 1973.1, 1974, 1974.1, 1975, 1975.1, 1976, 1976.1, 1977, 1977.1, 1978, 1978.1, 1979, 1979.1, 1980, 1980.1, 1981, 1981.1, 1982, 1982.1, 1983, 1983.1, 1984, 1984.1, 1985, 1985.1, 1986, 1986.1, 1987, 1987.1, 1988, 1988.1, 1989, 1989.1, 1990, 1990.1, 1991, 1991.1, 1992, 1992.1, 1993, 1993.1, 1994, 1994.1, 1995, 1995.1, 1996, 1996.1, 1997, 1997.1, 1998, 1998.1, 1999, 1999.1, 2000, 2000.1, 2001, 2001.1, 2002, 2002.1, 2003, 2003.1, 2004, 2004.1, 2005, 2005.1, 2006, 2006.1, 2007, 2007.1, 2008, 2008.1, 2009, 2009.1, 2010, 2010.1, 2011, 2011.1, 2012, 2012.1, 2013, 2013.1, 2014, 2014.1, 2015, 2015.1, 2016, 2016.1, 2017, 2017.1, 2018, 2018.1, ...]
Index: []

[0 rows x 110 columns]


### Lista países para padrão internacional

In [131]:
# pegar os países para aplicar tradução dos valores (registro para aplicar tradução )
# data_vinhos["País"].to_csv("out_dt/coutry_portuguese.csv")
# aplicado tradução dos valores e puxado valores em padrão internacional do csv country_pt
dt_country = pd.read_csv("out_dt\country_pt.csv")
dt_paises = pd.read_csv("out_dt\coutry_portuguese.csv")
print(dt_country.head(3))
print(dt_paises.head(3))

print(f"\n o tamanho dos paises a traudzidos ingles: {dt_country.shape[0]}, portugues: {dt_paises.shape[0]} ")


                        country
0                   Afghanistan
1                  South Africa
2  Germany, Democratic Republic
                              pais
0                      Afeganistão
1                    África do Sul
2  Alemanha, República Democrática

 o tamanho dos paises a traudzidos ingles: 136, portugues: 136 


<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\estev\AppData\Local\Temp\ipykernel_26208\1347148700.py:4: SyntaxWarning: invalid escape sequence '\c'
  dt_country = pd.read_csv("out_dt\country_pt.csv")
C:\Users\estev\AppData\Local\Temp\ipykernel_26208\1347148700.py:5: SyntaxWarning: invalid escape sequence '\c'
  dt_paises = pd.read_csv("out_dt\coutry_portuguese.csv")


### Filtrando para os dados pertencentes a análise (ultimos 15 anos )

In [132]:
# remover dois primeiros que não seguem padrão de separação entre valor e quantidade 
# ['Id', 'País', '1970', '1970.1', '1971', '1971.1', '1972', '1972.1',
columns_valor_quantidade_select = data_vinhos.columns[2:]

# separar os valores US$ das quantidades Kg Litros
columns_valor_select = [coluna for coluna in columns_valor_quantidade_select if ".1" in coluna]

# separa os valores de quantidade Kg Litros, são as colunas restantes 
columns_quant_select = [coluna for coluna in columns_valor_quantidade_select if ".1" not in coluna]

print(f"São {len(columns_valor_select)} colunas de valores US$ {columns_valor_select}\n ")
print(f"São {len(columns_quant_select)} colunas de quantidade Kg Litros {columns_quant_select}\n ")

# somente analisaremos os ultimos 15 anos 
dollar_15years = columns_valor_select[-15:]
quant_15years = columns_quant_select[-15:]
print(f"São {len(dollar_15years)} colunas de valores US$ {dollar_15years}\n ")
print(f"São {len(quant_15years)} colunas de quantidade Kg Litros {quant_15years}\n ")

São 54 colunas de valores US$ ['1970.1', '1971.1', '1972.1', '1973.1', '1974.1', '1975.1', '1976.1', '1977.1', '1978.1', '1979.1', '1980.1', '1981.1', '1982.1', '1983.1', '1984.1', '1985.1', '1986.1', '1987.1', '1988.1', '1989.1', '1990.1', '1991.1', '1992.1', '1993.1', '1994.1', '1995.1', '1996.1', '1997.1', '1998.1', '1999.1', '2000.1', '2001.1', '2002.1', '2003.1', '2004.1', '2005.1', '2006.1', '2007.1', '2008.1', '2009.1', '2010.1', '2011.1', '2012.1', '2013.1', '2014.1', '2015.1', '2016.1', '2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1', '2023.1']
 
São 54 colunas de quantidade Kg Litros ['1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '20

### Derretendo a tabela para formar dados longos (melt)

In [133]:
# transformaremos em dados longos dois dataframes um de valores e um de qauntidades (Kg)
# resultar uma tabela com valores repetidos de País em que uma sequencia de colunas vira duas colunas 
# uma coluna valor com os valores das colunas referente ao index chamado dollar e a coluna ano referente a 
# informação presente no nome da coluna e uma coluna nome 
melted_value = data_vinhos.melt(id_vars=['País'], value_vars=dollar_15years, var_name='ano', value_name='dollar_US$')
print(melted_value.head(6))
print(melted_value.shape)

                              País     ano  dollar_US$
0                      Afeganistão  2009.1           0
1                    África do Sul  2009.1           0
2  Alemanha, República Democrática  2009.1      393482
3                           Angola  2009.1       84235
4                         Anguilla  2009.1           0
5                Antígua e Barbuda  2009.1           0
(2055, 3)


In [134]:
# resultar uma tabela com valores repetidos de País em que uma sequencia de colunas vira duas colunas 
# uma coluna valor com os valores das colunas referente ao index chamado quant e a coluna ano referente a 
# informação presente no nome da coluna e uma coluna nome 
melted_quant = data_vinhos.melt(id_vars=['País'], value_vars=quant_15years, var_name='ano', value_name='quant_Kg_L')
print(melted_value.head(6))
print(melted_value.shape)

                              País     ano  dollar_US$
0                      Afeganistão  2009.1           0
1                    África do Sul  2009.1           0
2  Alemanha, República Democrática  2009.1      393482
3                           Angola  2009.1       84235
4                         Anguilla  2009.1           0
5                Antígua e Barbuda  2009.1           0
(2055, 3)


### Criando indexação semelhante para join entre os dados relacionados

In [135]:
melted_quant["pais_ano"] = melted_quant["País"] + "_" + melted_quant["ano"]
melted_quant.index = melted_quant['pais_ano']
melted_quant.head(5)

,País,ano,quant_Kg_L,pais_ano
pais_ano,,,,
Afeganistão_2009,Afeganistão,2009,0,Afeganistão_2009
África do Sul_2009,África do Sul,2009,0,África do Sul_2009
"Alemanha, República Democrática_2009","Alemanha, República Democrática",2009,225086,"Alemanha, República Democrática_2009"
Angola_2009,Angola,2009,54786,Angola_2009
Anguilla_2009,Anguilla,2009,0,Anguilla_2009


In [136]:
melted_value["pais_ano"] = melted_value["País"] + "_" + melted_value["ano"].str[:-2]
melted_value.index = melted_value['pais_ano']
melted_value.head(5)

,País,ano,dollar_US$,pais_ano
pais_ano,,,,
Afeganistão_2009,Afeganistão,2009.1,0,Afeganistão_2009
África do Sul_2009,África do Sul,2009.1,0,África do Sul_2009
"Alemanha, República Democrática_2009","Alemanha, República Democrática",2009.1,393482,"Alemanha, República Democrática_2009"
Angola_2009,Angola,2009.1,84235,Angola_2009
Anguilla_2009,Anguilla,2009.1,0,Anguilla_2009


In [137]:
# unindo os dataframe por meio do index criado comum entre eles para bater valores, poderia ser por id de ordenação 
# para ajudar na ordem e averiguação dos dados foi escolhido o novo index de relação 
join_value_quant = melted_value.join(other=melted_quant, on=None, lsuffix='_left', rsuffix='', how='left')
join_value_quant.head(3)

,País_left,ano_left,dollar_US$,pais_ano_left,País,ano,quant_Kg_L,pais_ano
pais_ano,,,,,,,,
Afeganistão_2009,Afeganistão,2009.1,0,Afeganistão_2009,Afeganistão,2009,0,Afeganistão_2009
África do Sul_2009,África do Sul,2009.1,0,África do Sul_2009,África do Sul,2009,0,África do Sul_2009
"Alemanha, República Democrática_2009","Alemanha, República Democrática",2009.1,393482,"Alemanha, República Democrática_2009","Alemanha, República Democrática",2009,225086,"Alemanha, República Democrática_2009"


In [138]:
# removendo informações desnecessárias auxiliares e limpando 
# remover os dados repetidos de left 
select_remove_left = [nome_coluna for nome_coluna in join_value_quant if '_left' not in nome_coluna]
join_value_quant = join_value_quant[select_remove_left]
print(join_value_quant.head(3))
print("\n")
# removendo coluna pais_ano e reindexando 
select_remove_column_pais_ano = [nome_coluna for nome_coluna in join_value_quant if 'pais_ano' not in nome_coluna]
# join_value_quant.drop(columns=['pais_ano'], inplace=True) opção, porem não aceita duas remoções .. 
join_value_quant = join_value_quant[select_remove_column_pais_ano]
join_value_quant.index = range(join_value_quant.shape[0])
join_value_quant.head(4)



                                      dollar_US$  \
pais_ano                                           
Afeganistão_2009                               0   
África do Sul_2009                             0   
Alemanha, República Democrática_2009      393482   

                                                                 País   ano  \
pais_ano                                                                      
Afeganistão_2009                                          Afeganistão  2009   
África do Sul_2009                                      África do Sul  2009   
Alemanha, República Democrática_2009  Alemanha, República Democrática  2009   

                                      quant_Kg_L  \
pais_ano                                           
Afeganistão_2009                               0   
África do Sul_2009                             0   
Alemanha, República Democrática_2009      225086   

                                                                  pais_ano  
pais_

,dollar_US$,País,ano,quant_Kg_L
0,0,Afeganistão,2009,0
1,0,África do Sul,2009,0
2,393482,"Alemanha, República Democrática",2009,225086
3,84235,Angola,2009,54786


### Perfumarias finais 

In [141]:
# renomeando colunas 
join_value_quant.columns = ['valor_us$', 'pais', 'ano', 'quant_kg']

# reordenando as colunas 
new_order = ['pais', 'ano', 'quant_kg','valor_us$']
dt_new_order = join_value_quant[new_order]

# adicionando a coluna de tipo de exportação 
dt_new_order['tipo'] = 'Vinhos de mesa'

final_transformation = dt_new_order
final_transformation.columns.name = 'id'
final_transformation.index = range(final_transformation.shape[0])
final_transformation

C:\Users\estev\AppData\Local\Temp\ipykernel_26208\279090920.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_new_order['tipo'] = 'Vinhos de mesa'


id,pais,ano,quant_kg,valor_us$,tipo
0,Afeganistão,2009,0,0,Vinhos de mesa
1,África do Sul,2009,0,0,Vinhos de mesa
2,"Alemanha, República Democrática",2009,225086,393482,Vinhos de mesa
3,Angola,2009,54786,84235,Vinhos de mesa
4,Anguilla,2009,0,0,Vinhos de mesa
...,...,...,...,...,...
2050,Tuvalu,2023,0,0,Vinhos de mesa
2051,Uruguai,2023,326093,454271,Vinhos de mesa
2052,Vanuatu,2023,0,0,Vinhos de mesa
2053,Venezuela,2023,141030,220512,Vinhos de mesa


In [142]:
final_transformation.to_csv("out_dt/ExpVinhoMesa.csv")

Connected to aula2amb (Python 3.12.2)